This file runs a gridsearch in order to find optimal parameters to build the auto encoder. The best parameters are saved to a file and retrieved by the main file. I ran the gridsearch a few times each time narrowing down the best parameters it found in order to come up with the current gridsearch parameters. With more time and computing power it would have been better to automate this process.

In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Standard import(s)
import numpy as np
import csv
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import pickle

# Keras import(s)
#if float(tf.__version__[:3])>2.8:
from tensorflow.keras.utils import plot_model
#from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.layers import BatchNormalization,Input, Dense, Dropout, Flatten, Reshape, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Lambda, UpSampling2D
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.losses import mae
from tensorflow.keras.losses import mean_squared_error 
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.backend import set_image_data_format
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
set_image_data_format('channels_last')  # Images are formatted as (N, N, 1), with colour-channels last

# Keras import(s)
if float(tf.__version__[:3])>2.8:
    from tensorflow.keras.utils import plot_model
else:
    from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras import regularizers
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
#Open file containing the saved SM training data 
with open('X_datafile.pickle', 'rb') as handlex:
    X_train = pickle.load(handlex)


In [3]:
#Auto encoder function used for gridsearch
def build_ae(num_dense_layers,first_dense_dim,regularization_factor ,act_func,latent_dim = 10):
    input_dim = X_train.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(first_dense_dim, activation="relu", activity_regularizer=regularizers.l1(regularization_factor))(input_layer)
    dim =first_dense_dim/2
    for i in range(num_dense_layers - 1):
        encoder = Dense(int(dim),activation="relu")(encoder)
        dim = dim/2
                        
    encoder = x = Dense(latent_dim, activation="relu")(encoder)
    decoder = encoder
    
    for i in range(num_dense_layers - 1):
        dim = dim*2
        decoder = Dense(int(dim), activation="relu")(decoder)
    decoder = Dense(first_dense_dim, activation="relu")(decoder)
    decoder = Dense(input_dim, activation=act_func)(decoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder



In [4]:
#Define dictionary of parameters to use in gridsearch.
param_grid = {
    'first_dense_dim': [256,128],
    'num_dense_layers': [1,2,3],
    'regularization_factor': [0.0001],
    'act_func' : ['linear','LeakyReLU']
 
}


In [5]:
#Create an estimator to be used for the gridsearh
ae_estimator = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_ae, epochs=5)

#execute gridsearch, using cv = 3 to minimise the searching time
grid = GridSearchCV(ae_estimator, param_grid,scoring='neg_mean_squared_error',cv = 3)      #negative MSE to minimise reconstruction error

C:\Users\Theo\AppData\Local\Temp\ipykernel_15048\4264055899.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ae_estimator = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_ae, epochs=5)


In [6]:
grid.fit(X_train, X_train, batch_size = 100, shuffle = True, validation_split = 0.2)
best_params = grid.best_params_  #get the best parameters found by the gridsearch 

Train on 145180 samples, validate on 36296 samples
Epoch 1/5
143300/145180 [============================>.] - ETA: 0s - loss: 0.0159

C:\Users\Theo\Anaconda3\envs\daml\lib\site-packages\keras\engine\training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


145180/145180 [==============================] - 5s 36us/sample - loss: 0.0158 - val_loss: 0.0079
Epoch 2/5
145180/145180 [==============================] - 5s 32us/sample - loss: 0.0072 - val_loss: 0.0061
Epoch 3/5
145180/145180 [==============================] - 5s 36us/sample - loss: 0.0058 - val_loss: 0.0055
Epoch 4/5
145180/145180 [==============================] - 5s 31us/sample - loss: 0.0054 - val_loss: 0.0053
Epoch 5/5
145180/145180 [==============================] - 4s 28us/sample - loss: 0.0052 - val_loss: 0.0051


C:\Users\Theo\Anaconda3\envs\daml\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 145180 samples, validate on 36296 samples
Epoch 1/5
143800/145180 [============================>.] - ETA: 0s - loss: 0.0147

...

145180/145180 [==============================] - 12s 85us/sample - loss: 0.0162 - val_loss: 0.0091
Epoch 2/5
145180/145180 [==============================] - 10s 72us/sample - loss: 0.0081 - val_loss: 0.0075
Epoch 3/5
145180/145180 [==============================] - 10s 72us/sample - loss: 0.0072 - val_loss: 0.0071
Epoch 4/5
145180/145180 [==============================] - 10s 72us/sample - loss: 0.0068 - val_loss: 0.0067
Epoch 5/5
145180/145180 [==============================] - 10s 68us/sample - loss: 0.0066 - val_loss: 0.0066


C:\Users\Theo\Anaconda3\envs\daml\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 217771 samples, validate on 54443 samples
Epoch 1/5
217000/217771 [============================>.] - ETA: 0s - loss: 0.0120

C:\Users\Theo\Anaconda3\envs\daml\lib\site-packages\keras\engine\training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


217771/217771 [==============================] - 15s 70us/sample - loss: 0.0119 - val_loss: 0.0072
Epoch 2/5
217771/217771 [==============================] - 13s 61us/sample - loss: 0.0068 - val_loss: 0.0064
Epoch 3/5
217771/217771 [==============================] - 13s 62us/sample - loss: 0.0059 - val_loss: 0.0057
Epoch 4/5
217771/217771 [==============================] - 13s 61us/sample - loss: 0.0056 - val_loss: 0.0055
Epoch 5/5
217771/217771 [==============================] - 13s 61us/sample - loss: 0.0045 - val_loss: 0.0043


In [9]:
#save the best parameters
with open('best_params.pickle', 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [10]:
print(best_params)

{'act_func': 'linear', 'first_dense_dim': 256, 'num_dense_layers': 1, 'regularization_factor': 0.0001}
